### **Format Dataset for YOLOv5**

In [1]:
import os
import shutil
import random
from tqdm import tqdm

# Define the base directory containing the SH17 dataset
base_dir = "/kaggle/input/sh17-dataset-for-ppe-detection"

# Paths to the train and validation file lists
train_file = os.path.join(base_dir, "train_files.txt")
val_file = os.path.join(base_dir, "val_files.txt")

# Directories for images and labels in the source dataset
images_dir = os.path.join(base_dir, "images")
labels_dir = os.path.join(base_dir, "labels")

# Output directory for prepared data
output_dir = "/kaggle/working/data"
os.makedirs(output_dir, exist_ok=True)

# Directories for train, validation, and test sets (images and labels)
train_images_dir = os.path.join(output_dir, "images/train")
val_images_dir = os.path.join(output_dir, "images/val")
test_images_dir = os.path.join(output_dir, "images/test")

train_labels_dir = os.path.join(output_dir, "labels/train")
val_labels_dir = os.path.join(output_dir, "labels/val")
test_labels_dir = os.path.join(output_dir, "labels/test")

# Create all required directories if they do not exist
for dir_path in [train_images_dir, val_images_dir, test_images_dir,
                 train_labels_dir, val_labels_dir, test_labels_dir]:
    os.makedirs(dir_path, exist_ok=True)

def copy_files(file_list, dest_images_dir, dest_labels_dir):
    """
    Copies image and corresponding label files to the specified directories.

    Args:
        file_list (list): List of file names to be copied.
        dest_images_dir (str): Destination directory for images.
        dest_labels_dir (str): Destination directory for labels.
    """
    for file in tqdm(file_list):
        file = file.strip()

        # Copy the image file
        image_path = os.path.join(images_dir, file)
        if os.path.exists(image_path):
            shutil.copy(image_path, dest_images_dir)
        else:
            print(f"Image not found: {image_path}")

        # Copy the corresponding label file
        label_path = os.path.join(labels_dir, file.replace(
            image_path.split(".")[-1], "txt"))
        if os.path.exists(label_path):
            shutil.copy(label_path, dest_labels_dir)
        else:
            print(f"Label not found: {label_path}")

# Read the list of training files from train_files.txt
with open(train_file, 'r') as f:
    train_files = f.readlines()

# Shuffle and split the training data into 80% training and 20% validation
random.shuffle(train_files)
split_idx = int(0.8 * len(train_files))
train_split = train_files[:split_idx]
val_split = train_files[split_idx:]

# Copy the training split files
print("Copying train files...")
copy_files(train_split, train_images_dir, train_labels_dir)

# Copy the validation split files
print("Copying val files...")
copy_files(val_split, val_images_dir, val_labels_dir)

# Copy the test files listed in val_files.txt
with open(val_file, 'r') as f:
    test_files = f.readlines()

print("Copying test files...")
copy_files(test_files, test_images_dir, test_labels_dir)

print("Data prepared!")

Copying train files...


100%|██████████| 5183/5183 [03:19<00:00, 26.02it/s]


Copying val files...


100%|██████████| 1296/1296 [00:51<00:00, 25.14it/s]


Copying test files...


100%|██████████| 1620/1620 [00:58<00:00, 27.59it/s]

Data prepared!


### **Setup environment for YOLOv5**

In [2]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17129, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 17129 (delta 43), reused 14 (delta 14), pack-reused 17067 (from 3)
Receiving objects: 100% (17129/17129), 15.84 MiB | 31.67 MiB/s, done.
Resolving deltas: 100% (11747/11747), done.
/kaggle/working/yolov5


In [3]:
!pip install -r requirements.txt
!pip install -q comet_ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.9/906.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.6/710.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.3/980.3 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.3 MB/s eta 0:00:00


### **Create YAML file**

In [4]:
import yaml

# Define the data to be written into the YAML file
data = {
    "names": [
        "Person", "Ear", "Earmuffs", "Face", "Face-guard", "Face-mask-medical",
        "Foot", "Tools", "Glasses", "Gloves", "Helmet", "Hands", "Head",
        "Medical-suit", "Shoes", "Safety-suit", "Safety-vest"
    ],
    "nc": 17,
    "test": "/kaggle/working/data/images/test",
    "train": "/kaggle/working/data/images/train",
    "val": "/kaggle/working/data/images/val"
}

# Define the output path for the YAML file
yaml_file_path = "../yolo.yaml"

# Write the data to the YAML file
with open(yaml_file_path, "w") as yaml_file:
    yaml.dump(data, yaml_file, default_flow_style=False)

print(f"YAML file has been created: {yaml_file_path}")

YAML file has been created: ../yolo.yaml


### **Update AMP Autocast Syntax in train.py**

In [5]:
# Define the file path for the script to be updated
file_path = "/kaggle/working/yolov5/train.py"

# Read the content of the file line by line
with open(file_path, "r") as file:
    lines = file.readlines()

# Initialize a list to store updated lines
updated_lines = []

# Iterate through each line and update specific syntax
for line in lines:
    if "with torch.cuda.amp.autocast(amp):" in line:
        # Replace the deprecated torch.cuda.amp.autocast syntax with the new syntax
        line = line.replace(
            "with torch.cuda.amp.autocast(amp):",
            "with torch.amp.autocast(device_type='cuda', dtype=torch.float16):"
        )
    # Add the updated line to the list
    updated_lines.append(line)

# Write the modified content back to the file
with open(file_path, "w") as file:
    file.writelines(updated_lines)

# Notify the user that the updates have been applied
print("The deprecated `torch.cuda.amp.autocast` warnings in train.py have been updated!")


The deprecated `torch.cuda.amp.autocast` warnings in train.py have been updated!


### **Train model YOLOv5**

In [6]:
!python train.py --img 640 --epochs 10 --batch-size 16 --data ../yolo.yaml --weights yolov5m.pt --project /kaggle/working/results/yolov5/train

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-01-13 16:47:02.660718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-13 16:47:02.857025: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-13 16:47:02.918932: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to regi

### **Evaluate on validation set**

In [7]:
!python val.py --weights /kaggle/working/results/yolov5/train/exp/weights/best.pt \
               --data ../yolo.yaml \
               --img 640 \
               --batch-size 16 \
               --project /kaggle/working/results/yolov5/val

val: data=../yolo.yaml, weights=['/kaggle/working/results/yolov5/train/exp/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/kaggle/working/results/yolov5/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-395-g6420a1db Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

Fusing layers... 
Model summary: 212 layers, 20917590 parameters, 0 gradients, 48.1 GFLOPs
val: Scanning /kaggle/working/data/labels/val.cache... 1296 images, 0 background
                 Class     Images  Instances          P          R      mAP50   
                   all       1296      11701      0.801      0.422      0.472      0.283
                Person       1296       2154      0.884      0.864      0.908       0.69
                   Ear       1296       1167      0.857       

### **Evaluate on test set**

In [8]:
!python val.py --weights /kaggle/working/results/yolov5/train/exp/weights/best.pt \
               --data ../yolo.yaml \
               --img 640 \
               --batch-size 16 \
               --tasks test \
               --project /kaggle/working/results/yolov5/test

usage: val.py [-h] [--data DATA] [--weights WEIGHTS [WEIGHTS ...]] [--batch-size BATCH_SIZE]
              [--imgsz IMGSZ] [--conf-thres CONF_THRES] [--iou-thres IOU_THRES]
              [--max-det MAX_DET] [--task TASK] [--device DEVICE] [--workers WORKERS]
              [--single-cls] [--augment] [--verbose] [--save-txt] [--save-hybrid] [--save-conf]
              [--save-json] [--project PROJECT] [--name NAME] [--exist-ok] [--half] [--dnn]
val.py: error: unrecognized arguments: --tasks test
